In [ ]:
import numpy as np
import pandas as pd

#1. IPO 당시의 기업 가치/ Acq 당시의 회사 가격으로 rank(1~4) 설정하기

In [ ]:
# acquisitions, ipos csv 불러오기
df_1_Acq=pd.read_csv('./Dataset/crunchbase_1/acquisitions.csv')
df_2_IPO=pd.read_csv('./Dataset/crunchbase_2/ipos.csv')

# 기업 가치 기준으로 오름차순 정렬
Acquired=df_1_Acq[['acquiree_uuid','price_usd']]
Acquired.dropna(inplace=True)
Acquired['price_usd'] = Acquired['price_usd'].apply(lambda x: int(x))
Acquired.sort_values(by=['price_usd'], inplace=True)

IPO=df_2_IPO[['org_uuid','valuation_price_usd']]
IPO.dropna(inplace=True)
IPO['valuation_price_usd'] = IPO['valuation_price_usd'].apply(lambda x: int(x))
IPO.sort_values(by=['valuation_price_usd'], inplace=True)

# Rank column 생성
Acquired['Rank_Acq']=0
IPO['Rank_IPO']=0

# 4개의 rank에 같은 수의 기업이 포함되도록 rank 부여

n=0
for rank in range (1,5):
  for i in range(int(len(Acquired)/4)):
    Acquired['Rank_Acq'].iloc[n]=rank
    n+=1
  
Acquired['Rank_Acq'].iloc[-1]=4
Acquired['Rank_Acq'].iloc[-2]=4

n=0
for rank in range (1,5):
  for i in range(int(len(IPO)/4)):
    IPO['Rank_IPO'].iloc[n]=rank
    n+=1
IPO['Rank_IPO'].iloc[-1]=4

#uuid와 rank 정보만 추출
Acquired=Acquired[['Rank_Acq','acquiree_uuid']]
IPO=IPO[['Rank_IPO','org_uuid']]

# 하나의 데이터 프레임으로 merge(uuid)
Acquired=Acquired.rename(columns={'acquiree_uuid':'org_uuid'})
Ranks=pd.merge(IPO,Acquired,how='outer',on='org_uuid')

#csv 파일로 저장
Ranks.to_csv('./Dataset/Ranks.csv') 


/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
<ipython-input-14-36ab84f58844>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Acquired['price_usd'] = Acquired['price_usd'].apply(lambda x: int(x))
<ipython-input-14-36ab84f58844>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

#2. (IPO 당시 회사 기업 가치/ 인수 합병 시 회사 가격)과,각 funding round에서의 회사 가치  를 비교하여 회사 가치 성장 정도 계산하기

IPO 당시 회사 기업 가치/ 인수 합병 시 회사 가격 을 전체 회사에서 z 값으로 구하고
각 funding round에서의 회사 가치를 전체 회사에서 z값으로 구한 후
이 차이를 구하여 z 값 차이가 높은 순 rank 1~4 부여

##z값 계산(정규화 하기)

In [ ]:
# Data Import


df_1_funding_rounds=pd.read_csv('./Dataset/crunchbase_1/funding_rounds.csv')

df_1_Acq=pd.read_csv('./Dataset/crunchbase_1/acquisitions.csv')
df_2_IPO=pd.read_csv('./Dataset/crunchbase_2/ipos.csv')


#Funding Round 별로 데이터 구분

SeriesA = df_1_funding_rounds[df_1_funding_rounds['investment_type']=='series_a']
SeriesB = df_1_funding_rounds[df_1_funding_rounds['investment_type']=='series_b']
SeriesC = df_1_funding_rounds[df_1_funding_rounds['investment_type']=='series_c']
Angel = df_1_funding_rounds[df_1_funding_rounds['investment_type']=='angel']
PreSeed = df_1_funding_rounds[df_1_funding_rounds['investment_type']=='pre_seed']
Seed = df_1_funding_rounds[df_1_funding_rounds['investment_type']=='seed']

#post_money_valuation_usd 순으로 sort, post_money_valuation_usd에 대한 결측값 제거

SeriesA = SeriesA.sort_values(by=['post_money_valuation_usd'])
SeriesA = SeriesA.drop(SeriesA[SeriesA['post_money_valuation_usd'].isnull()].index)

SeriesB = SeriesB.sort_values(by=['post_money_valuation_usd'])
SeriesB = SeriesB.drop(SeriesB[SeriesB['post_money_valuation_usd'].isnull()].index)

SeriesC = SeriesC.sort_values(by=['post_money_valuation_usd'])
SeriesC = SeriesC.drop(SeriesC[SeriesC['post_money_valuation_usd'].isnull()].index)

Angel = Angel.sort_values(by=['post_money_valuation_usd'])
Angel = Angel.drop(Angel[Angel['post_money_valuation_usd'].isnull()].index)

PreSeed = PreSeed.sort_values(by=['post_money_valuation_usd'])
PreSeed = PreSeed.drop(PreSeed[PreSeed['post_money_valuation_usd'].isnull()].index)

Seed = Seed.sort_values(by=['post_money_valuation_usd'])
Seed = Seed.drop(Seed[Seed['post_money_valuation_usd'].isnull()].index)

#Seed, Angel, Preseed 묶어서 Seed로 할당
Seed = pd.concat([Seed,Angel,PreSeed],axis=0)
Seed = Seed.sort_values(by=['post_money_valuation_usd'])

#기업 가격/ 기업 가치 순으로 sort,결측값 제거

Acquired=df_1_Acq[['acquiree_uuid','price_usd']]
Acquired.dropna(inplace=True)
Acquired.sort_values(by=['price_usd'],inplace=True)

IPO=df_2_IPO[['org_uuid','valuation_price_usd']]
IPO.dropna(inplace=True)
IPO.sort_values(by=['valuation_price_usd'],inplace=True)

#z 값 계산(x-mean/std)
mean_Seed=Seed['post_money_valuation_usd'].mean()
dev_Seed=Seed['post_money_valuation_usd'].std()
Seed['zSeed']=(Seed['post_money_valuation_usd']-mean_Seed)/dev_Seed
Seed=Seed.rename(columns={'post_money_valuation_usd':'post_money_valuation_usd_Seed','raised_amount_usd':'raised_amount_usd_Seed'})
Seed=Seed[['org_uuid','raised_amount_usd_Seed','zSeed']]

mean_A=SeriesA['post_money_valuation_usd'].mean()
dev_Seed=SeriesA['post_money_valuation_usd'].std()
SeriesA['zA']=(SeriesA['post_money_valuation_usd']-mean_A)/dev_Seed
SeriesA=SeriesA.rename(columns={'post_money_valuation_usd':'post_money_valuation_usd_A','raised_amount_usd':'raised_amount_usd_A'})
SeriesA=SeriesA[['org_uuid','raised_amount_usd_A','zA']]

mean_B=SeriesB['post_money_valuation_usd'].mean()
dev_B=SeriesB['post_money_valuation_usd'].std()
SeriesB['zB']=(SeriesB['post_money_valuation_usd']-mean_B)/dev_B
SeriesB=SeriesB.rename(columns={'post_money_valuation_usd':'post_money_valuation_usd_B','raised_amount_usd':'raised_amount_usd_B'})
SeriesB=SeriesB[['org_uuid','raised_amount_usd_B','zB']]

mean_C=SeriesC['post_money_valuation_usd'].mean()
dev_C=SeriesC['post_money_valuation_usd'].std()
SeriesC['zC']=(SeriesC['post_money_valuation_usd']-mean_C)/dev_C
SeriesC=SeriesC.rename(columns={'post_money_valuation_usd':'post_money_valuation_usd_C','raised_amount_usd':'raised_amount_usd_C'})
SeriesC=SeriesC[['org_uuid','raised_amount_usd_C','zC']]


mean_Acq=Acquired['price_usd'].mean()
dev_Acq=Acquired['price_usd'].std()
Acquired['zAcq']=(Acquired['price_usd']-mean_Acq)/dev_Acq
Acquired=Acquired[['acquiree_uuid','zAcq']]
Acquired=Acquired.rename(columns={'acquiree_uuid':'org_uuid'})

mean_IPO=IPO['valuation_price_usd'].mean()
dev_IPO=IPO['valuation_price_usd'].std()
IPO['zIPO']=(IPO['valuation_price_usd']-mean_IPO)/dev_IPO
IPO=IPO[['org_uuid','zIPO']]


Zs=pd.merge(SeriesA,SeriesB,how='outer',on='org_uuid')
Zs=pd.merge(Zs,SeriesC,how='outer',on='org_uuid')
Zs=pd.merge(Zs,Seed,how='outer',on='org_uuid')
Zs=pd.merge(Zs,Acquired,how='outer',on='org_uuid')
Zs=pd.merge(Zs,IPO,how='outer',on='org_uuid')



#Zs의 차이 계산
Zs_Acq=Zs.copy()
Zs_Acq['SeedDelta']=Zs_Acq['zAcq']-Zs_Acq['zSeed']
Zs_Acq['ADelta']=Zs_Acq['zAcq']-Zs_Acq['zA']
Zs_Acq['BDelta']=Zs_Acq['zAcq']-Zs_Acq['zB']
Zs_Acq['CDelta']=Zs_Acq['zAcq']-Zs_Acq['zC']

Zs_IPO=Zs.copy()
Zs_IPO['SeedDelta']=Zs_IPO['zIPO']-Zs_IPO['zSeed']
Zs_IPO['ADelta']=Zs_IPO['zIPO']-Zs_IPO['zA']
Zs_IPO['BDelta']=Zs_IPO['zIPO']-Zs_IPO['zB']
Zs_IPO['CDelta']=Zs_IPO['zIPO']-Zs_IPO['zC']

#funding round 별 post_nomy_valuationd의 컬럼명 리스트에 할당
post_money_valuations=['post_money_valuation_usd_Before','post_money_valuation_usd_Seed','post_money_valuation_usd_A','post_money_valuation_usd_B','post_money_valuation_usd_C']

/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
<ipython-input-4-dc285cc16c44>:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Acquired['zAcq']=(Acquired['price_usd']-mean_Acq)/dev_Acq
<ipython-input-4-dc285cc16c44>:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexi

##Rank 매기기

##IPO

In [ ]:
#1 Seed
df_Ser_1=Zs_IPO.copy()
df_Ser_1 = df_Ser_1.drop(df_Ser_1[df_Ser_1['SeedDelta'].isnull()].index)
df_Ser_1['Rank_Seed']=0
Ser_Delta='SeedDelta'

#Delta 값 양수인 것과 음수 인 것의 개수 구하기
Rank3=len(df_Ser_1[df_Ser_1['SeedDelta']>=0])
Rank2=len(df_Ser_1[df_Ser_1['SeedDelta']<=0])

#z값 양수: Rank 3, 4, z값 음수: Rank 1, 2 이며, 
#rank 3과 4 는 같은 개수, rank 1과 2는 같은 개수로 나눔

Rank_make_1=[]
df_Ser_1.sort_values(by='SeedDelta',ascending=False) #큰 것부터 나열(Rank4부터 차례로 z값 높은 것)

#Rank 할당
for num in range(len(df_Ser_1)):
  Del=df_Ser_1['SeedDelta'].iloc[num]
  if num<=Rank3/2:
    Rank_make_1.append(4)
  elif Del>=0:
    Rank_make_1.append(3)
  elif num<(Rank3+Rank2/2):
    Rank_make_1.append(2)
  else:
    Rank_make_1.append(1)

#IPO자료에 org_uuid를 기준으로 rank값 병합
df_Ser_1['Rank_Seed']=Rank_make_1
Zs_IPO_=pd.merge(Zs_IPO,df_Ser_1,how='outer',on='org_uuid')

#2.SeriesA

df_Ser_2=Zs_IPO.copy()
df_Ser_2 = df_Ser_2.drop(df_Ser_2[df_Ser_2['ADelta'].isnull()].index)
df_Ser_2['Rank_A']=0
Ser_Delta='ADelta'

Rank3=len(df_Ser_2[df_Ser_2['ADelta']>=0])
Rank2=len(df_Ser_2[df_Ser_2['ADelta']<=0])
Rank_make_2=[]
df_Ser_2.sort_values(by='ADelta',ascending=False) #큰 것부터 나열

for num in range(len(df_Ser_2)):
  Del=df_Ser_2['ADelta'].iloc[num]
  if num<=Rank3/2:
    Rank_make_2.append(4)
  elif Del>=0:
    Rank_make_2.append(3)
  elif num<(Rank3+Rank2/2):
    Rank_make_2.append(2)
  else:
    Rank_make_2.append(1)


df_Ser_2['Rank_A']=Rank_make_2
Zs_IPO_=pd.merge(Zs_IPO_,df_Ser_2,how='outer',on='org_uuid')

#3. SeriesB
df_Ser_3=Zs_IPO.copy()
df_Ser_3 = df_Ser_3.drop(df_Ser_3[df_Ser_3['BDelta'].isnull()].index)

df_Ser_3['Rank_B']=0
Ser_Delta='BDelta'

Rank3=len(df_Ser_3[df_Ser_3['BDelta']>=0])
Rank2=len(df_Ser_3[df_Ser_3['BDelta']<=0])


Rank_make_3=[]
df_Ser_3.sort_values(by='BDelta',ascending=False) #큰 것부터 나열
for num in range(len(df_Ser_3)):
  Del=df_Ser_3['BDelta'].iloc[num]
  if num<=Rank3/2:
    Rank_make_3.append(4)
  elif Del>=0:
    Rank_make_3.append(3)
  elif num<(Rank3+Rank2/2):
    Rank_make_3.append(2)
  else:
    Rank_make_3.append(1)



df_Ser_3['Rank_B']=Rank_make_3
Zs_IPO_=pd.merge(Zs_IPO_,df_Ser_3,how='outer',on='org_uuid')

#4. SeriesC
df_Ser_4=Zs_IPO.copy()
df_Ser_4 = df_Ser_4.drop(df_Ser_4[df_Ser_4['CDelta'].isnull()].index)
df_Ser_4['Rank_C']=0
Ser_Delta='CDelta'

Rank3=len(df_Ser_4[df_Ser_4['CDelta']>=0])
Rank2=len(df_Ser_4[df_Ser_4['CDelta']<=0])
Rank_make_3=[]
df_Ser_4.sort_values(by='CDelta',ascending=False) #큰 것부터 나열

for num in range(len(df_Ser_4)):
  Del=df_Ser_4['CDelta'].iloc[num]
  if num<=Rank3/2:
    Rank_make_3.append(4)
  elif Del>=0:
    Rank_make_3.append(3)
  elif num<(Rank3+Rank2/2):
    Rank_make_3.append(2)
  else:
    Rank_make_3.append(1)



df_Ser_4['Rank_C']=Rank_make_3
Zs_IPO_=pd.merge(Zs_IPO_,df_Ser_4,how='outer',on='org_uuid')

<ipython-input-5-01fabca6b976>:87: FutureWarning: Passing 'suffixes' which cause duplicate columns {'zSeed_x', 'CDelta_x', 'ADelta_x', 'zB_x', 'zIPO_x', 'BDelta_x', 'raised_amount_usd_B_x', 'zAcq_x', 'SeedDelta_x', 'raised_amount_usd_A_x', 'zC_x', 'zA_x', 'raised_amount_usd_C_x', 'raised_amount_usd_Seed_x'} in the result is deprecated and will raise a MergeError in a future version.
  Zs_IPO_=pd.merge(Zs_IPO_,df_Ser_3,how='outer',on='org_uuid')


In [ ]:
#각 랭크마다 개수
Ranks_list=["Rank_A","Rank_B","Rank_C","Rank_Seed"]
for i in Ranks_list:
  j=Zs_IPO_[i].dropna(inplace=False)
  print(len(j))

369
167
178
382


In [ ]:
Zs_IPO_.to_csv("./Dataset/Zs_IPO.csv", index = False)

##Acq

In [ ]:
#1 Seed
df_Ser_1=Zs_Acq.copy()
df_Ser_1 = df_Ser_1.drop(df_Ser_1[df_Ser_1['SeedDelta'].isnull()].index)
df_Ser_1['Rank_Seed']=0
Ser_Delta='SeedDelta'

#Delta 값 양수인 것과 음수 인 것의 개수 구하기
Rank3=len(df_Ser_1[df_Ser_1['SeedDelta']>=0])
Rank2=len(df_Ser_1[df_Ser_1['SeedDelta']<=0])

#z값 양수: Rank 3, 4, z값 음수: Rank 1, 2 이며, 
#rank 3과 4 는 같은 개수, rank 1과 2는 같은 개수로 나눔

Rank_make_1=[]
df_Ser_1.sort_values(by='SeedDelta',ascending=False) #큰 것부터 나열(Rank4부터 차례로 z값 높은 것)

#Rank 할당
for num in range(len(df_Ser_1)):
  Del=df_Ser_1['SeedDelta'].iloc[num]
  if num<=Rank3/2:
    Rank_make_1.append(4)
  elif Del>=0:
    Rank_make_1.append(3)
  elif num<(Rank3+Rank2/2):
    Rank_make_1.append(2)
  else:
    Rank_make_1.append(1)

#IPO자료에 org_uuid를 기준으로 rank값 병합
df_Ser_1['Rank_Seed']=Rank_make_1
Zs_Acq_=pd.merge(Zs_Acq,df_Ser_1,how='outer',on='org_uuid')

#2.SeriesA

df_Ser_2=Zs_Acq.copy()
df_Ser_2 = df_Ser_2.drop(df_Ser_2[df_Ser_2['ADelta'].isnull()].index)
df_Ser_2['Rank_A']=0
Ser_Delta='ADelta'

Rank3=len(df_Ser_2[df_Ser_2['ADelta']>=0])
Rank2=len(df_Ser_2[df_Ser_2['ADelta']<=0])
Rank_make_2=[]
df_Ser_2.sort_values(by='ADelta',ascending=False) #큰 것부터 나열

for num in range(len(df_Ser_2)):
  Del=df_Ser_2['ADelta'].iloc[num]
  if num<=Rank3/2:
    Rank_make_2.append(4)
  elif Del>=0:
    Rank_make_2.append(3)
  elif num<(Rank3+Rank2/2):
    Rank_make_2.append(2)
  else:
    Rank_make_2.append(1)


df_Ser_2['Rank_A']=Rank_make_2
Zs_Acq_=pd.merge(Zs_Acq_,df_Ser_2,how='outer',on='org_uuid')

#3. SeriesB
df_Ser_3=Zs_Acq.copy()
df_Ser_3 = df_Ser_3.drop(df_Ser_3[df_Ser_3['BDelta'].isnull()].index)

df_Ser_3['Rank_B']=0
Ser_Delta='BDelta'

Rank3=len(df_Ser_3[df_Ser_3['BDelta']>=0])
Rank2=len(df_Ser_3[df_Ser_3['BDelta']<=0])


Rank_make_3=[]
df_Ser_3.sort_values(by='BDelta',ascending=False) #큰 것부터 나열
for num in range(len(df_Ser_3)):
  Del=df_Ser_3['BDelta'].iloc[num]
  if num<=Rank3/2:
    Rank_make_3.append(4)
  elif Del>=0:
    Rank_make_3.append(3)
  elif num<(Rank3+Rank2/2):
    Rank_make_3.append(2)
  else:
    Rank_make_3.append(1)



df_Ser_3['Rank_B']=Rank_make_3
Zs_Acq_=pd.merge(Zs_Acq_,df_Ser_3,how='outer',on='org_uuid')

#4. SeriesC
df_Ser_4=Zs_Acq.copy()
df_Ser_4 = df_Ser_4.drop(df_Ser_4[df_Ser_4['CDelta'].isnull()].index)
df_Ser_4['Rank_C']=0
Ser_Delta='CDelta'

Rank3=len(df_Ser_4[df_Ser_4['CDelta']>=0])
Rank2=len(df_Ser_4[df_Ser_4['CDelta']<=0])
Rank_make_3=[]
df_Ser_4.sort_values(by='CDelta',ascending=False) #큰 것부터 나열

for num in range(len(df_Ser_4)):
  Del=df_Ser_4['CDelta'].iloc[num]
  if num<=Rank3/2:
    Rank_make_3.append(4)
  elif Del>=0:
    Rank_make_3.append(3)
  elif num<(Rank3+Rank2/2):
    Rank_make_3.append(2)
  else:
    Rank_make_3.append(1)



df_Ser_4['Rank_C']=Rank_make_3
Zs_Acq_=pd.merge(Zs_Acq_,df_Ser_4,how='outer',on='org_uuid')

<ipython-input-8-e0ecc63f5988>:87: FutureWarning: Passing 'suffixes' which cause duplicate columns {'zSeed_x', 'CDelta_x', 'ADelta_x', 'zB_x', 'zIPO_x', 'BDelta_x', 'raised_amount_usd_B_x', 'zAcq_x', 'SeedDelta_x', 'raised_amount_usd_A_x', 'zC_x', 'zA_x', 'raised_amount_usd_C_x', 'raised_amount_usd_Seed_x'} in the result is deprecated and will raise a MergeError in a future version.
  Zs_Acq_=pd.merge(Zs_Acq_,df_Ser_3,how='outer',on='org_uuid')


In [ ]:
#각 랭크마다 개수
Ranks_list=["Rank_A","Rank_B","Rank_C","Rank_Seed"]
for i in Ranks_list:
  j=Zs_Acq_[i].dropna(inplace=False)
  print(len(j))

386
364
92
463


In [ ]:
Zs_Acq_.to_csv("./Dataset/Zs_Acq.csv", index = False)